1. YOLO로 사람/총/칼 인식
2. Arcface로 얼굴 인식
3. 특정인에 대해서만! MediaPipe로 포즈 인식


In [8]:
#1. 얼굴 학습시키는 부분
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis

def initialize_arcface():
    app = FaceAnalysis(name="buffalo_l")  # ArcFace 모델 (buffalo_l은 기본 권장)
    app.prepare(ctx_id=-1, det_size=(640, 640))  # GPU: ctx_id=0, CPU: -1
    return app

def get_face_embedding(app, image_bgr):
    # ArcFace의 app.get()은 BGR 형식으로 이미지를 받기도 합니다.
    # 만약 RGB가 필요하면 cvtColor로 변환하세요.
    faces = app.get(image_bgr)
    if len(faces) > 0:
        return faces[0].embedding  # 첫 번째 얼굴의 임베딩
    else:
        return None

def generate_average_embedding(app, folder_path):
    embeddings = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"이미지 로드 실패: {img_path}")
                continue
            
            embedding = get_face_embedding(app, image)
            if embedding is not None:
                embeddings.append(embedding)
            else:
                print(f"얼굴 검출 실패: {img_path}")
    
    if len(embeddings) == 0:
        raise ValueError("임베딩을 하나도 생성하지 못했습니다.")
    
    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding

if __name__ == "__main__":
    app = initialize_arcface()
    # 내 얼굴 사진 폴더
    my_face_folder = "C:/Users/idea0/EE101/Jongsul/myface"  
    my_face_embedding = generate_average_embedding(app, my_face_folder)
    np.save("my_face_embedding.npy", my_face_embedding)  # 필요 시 저장
    print("내 얼굴 평균 임베딩 생성 완료.")

C:\python\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

C:\Users\idea0\AppData\Roaming\Python\Python38\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [7]:
import cv2
import mediapipe as mp
import time
import torch
import numpy as np

from ultralytics import YOLO
from sklearn.metrics.pairwise import cosine_similarity
from insightface.app import FaceAnalysis

#######################
# 1) 모델 초기화
#######################
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# YOLOv8 모델 로드
model = YOLO("yolov8n.pt").to(device)
# model = YOLO("C:/Users/idea0/EE101/Jongsul/Yolomodels/best.pt").to(device)

# ArcFace 초기화
arc_app = FaceAnalysis(name="buffalo_l")
arc_app.prepare(ctx_id=-1, det_size=(640, 640))  # CPU 사용 예시 (ctx_id=-1)

# 내 얼굴 임베딩 로드
my_face_embedding = np.load("my_face_embedding.npy")

def get_face_embedding(arc_app, face_img_bgr):
    faces = arc_app.get(face_img_bgr)
    if len(faces) == 0:
        return None
    return faces[0].embedding

def is_my_face(face_embedding, my_embedding, threshold=0.4):
    sim = cosine_similarity([face_embedding], [my_embedding])[0][0]
    return (sim > threshold), sim

#######################
# 2) Mediapipe Pose (단일 인물용)
#######################
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Pose 객체 (static_image_mode=True → 크롭된 이미지에 적합)
pose_estimator = mp_pose.Pose(
    static_image_mode=True,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 예: 왼/오른 팔 들기 판별용
LEFT_SHOULDER = 11
RIGHT_SHOULDER = 12
LEFT_WRIST = 15
RIGHT_WRIST = 16

def is_arm_raised(shoulder_y, wrist_y, threshold=0.05):
    """
    Mediapipe Pose에서 y=0은 화면 상단, 1은 하단(값이 작을수록 위).
    어깨 y좌표 - threshold보다 손목 y좌표가 작으면 '팔을 들었다'고 판단.
    """
    return wrist_y < (shoulder_y - threshold)

#######################
# 3) 웹캠 실행
#######################
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

prev_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽어올 수 없습니다.")
        break

    # YOLO로 사람 검출
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_yolo = model(rgb_frame)

    # YOLO 결과에서 사람(person)만 처리
    for box in results_yolo[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls)
        conf = float(box.conf)
        class_name = model.names[class_id]

        # 바운딩박스 표시
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{class_name}: {conf:.2f}"
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        if class_name.lower() == "person":
            # 사람 크롭
            person_crop = frame[y1:y2, x1:x2]
            if person_crop.size == 0:
                continue

            # ArcFace로 "내 얼굴"인지 판별
            face_embedding = get_face_embedding(arc_app, person_crop)
            if face_embedding is not None:
                same_person, sim = is_my_face(face_embedding, my_face_embedding, threshold=0.4)
                if same_person:
                    color = (0, 255, 0)
                    text_face = f"Me! (sim={sim:.2f})"
                else:
                    color = (0, 0, 255)
                    text_face = f"Not me (sim={sim:.2f})"

                cv2.putText(frame, text_face, (x1, y1 - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # **나**라면 Pose 인식
                if same_person:
                    # subimage를 RGB로 변환
                    sub_rgb = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)

                    # Mediapipe Pose 수행
                    pose_result = pose_estimator.process(sub_rgb)
                    if pose_result.pose_landmarks:
                        # 포즈 판별 (왼팔/오른팔)
                        landmarks = pose_result.pose_landmarks.landmark

                        # subimage width/height
                        sub_w = (x2 - x1)
                        sub_h = (y2 - y1)

                        left_shoulder_y = landmarks[LEFT_SHOULDER].y
                        right_shoulder_y = landmarks[RIGHT_SHOULDER].y
                        left_wrist_y = landmarks[LEFT_WRIST].y
                        right_wrist_y = landmarks[RIGHT_WRIST].y

                        left_arm_up = is_arm_raised(left_shoulder_y, left_wrist_y)
                        right_arm_up = is_arm_raised(right_shoulder_y, right_wrist_y)

                        if left_arm_up and right_arm_up:
                            action_text = "Both arms up"
                        elif left_arm_up:
                            action_text = "Left arm up"
                        elif right_arm_up:
                            action_text = "Right arm up"
                        else:
                            action_text = "Arms down"

                        # 결과 표시
                        cv2.putText(frame, action_text, (x1, y1 - 60),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)

                        # (옵션) 원본 프레임에 랜드마크 시각화
                        for i, lm in enumerate(landmarks):
                            # lm.x, lm.y는 [0..1] (subimage 기준)
                            px = x1 + int(lm.x * sub_w)
                            py = y1 + int(lm.y * sub_h)
                            cv2.circle(frame, (px, py), 3, (255, 0, 255), -1)

    # FPS 계산
    current_time = time.time()
    fps = 1.0 / (current_time - prev_time)
    prev_time = current_time
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Only My Pose", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

C:\Users\idea0\AppData\Roaming\Python\Python38\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


0: 480x640 1 person, 89.0ms
Speed: 2.0ms preprocess, 89.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



C:\Users\idea0\AppData\Roaming\Python\Python38\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


0: 480x640 1 person, 131.0ms
Speed: 2.0ms preprocess, 131.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



...

C:\Users\idea0\AppData\Roaming\Python\Python38\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 480x640 3 persons, 1 cell phone, 92.1ms
Speed: 15.0ms preprocess, 92.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


C:\Users\idea0\AppData\Roaming\Python\Python38\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
C:\Users\idea0\AppData\Roaming\Python\Python38\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 480x640 1 person, 95.0ms
Speed: 3.0ms preprocess, 95.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


C:\Users\idea0\AppData\Roaming\Python\Python38\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
